In [21]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

In [22]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [23]:
#Choosing the llm
#openai_llm = OpenAI(temperature=0.7)

from langchain.llms.bedrock import Bedrock
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from myutils import bedrock

os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
os.environ['AWS_PROFILE'] = 'sengledo+bedrock-benelux-Admin'
boto3_bedrock = bedrock.get_bedrock_client(os.environ.get('BEDROCK_ASSUME_ROLE', None))

model_kwargs={
        "maxTokenCount": 4096,
        "stopSequences": [],
        "temperature": 0,
        "topP": 1,
    }
titan_llm = Bedrock(model_id="amazon.titan-tg1-large", client=boto3_bedrock, model_kwargs=model_kwargs)
#titan_llm = Bedrock(model_id="anthropic.claude-v1", client=boto3_bedrock)



Create new client
  Using region: us-east-1
  Using profile: sengledo+bedrock-benelux-Admin
boto3 Bedrock client successfully created!


In [24]:
#q = "What type of mammal lays the biggest eggs?"
#q = "Is ostrich a mammal?"
#q = "Was Donald Trumph the president of USA in 2022?"
q = "The first woman to receive a Ph.D. in computer science was Dr. Barbara Liskov, who earned her degree from Stanford University in 1968."
llm = titan_llm

def generate_and_print(llm, q, table=None):
    total_prompt = """"""

    template = """Here is a statement:
    {statement}
    Make a bullet point list of the assumptions you made when given the above statement.\n\n"""
    prompt_template = PromptTemplate(input_variables=["statement"], template=template)
    assumptions_chain = LLMChain(llm=llm, prompt=prompt_template)
    total_prompt = total_prompt + template

    template = """Here is a bullet point list of assertions:
    {assertions}
    For each assertion, determine whether it is true or false. If it is false, explain why.\n\n"""
    prompt_template = PromptTemplate(input_variables=["assertions"], template=template)
    fact_checker_chain = LLMChain(llm=llm, prompt=prompt_template)
    total_prompt = total_prompt + template

    template = """Based on the above assertions, the final response is FALSE if one of the assertions is FALSE. Otherwise, TRUE. You should only respond with TRUE or FALSE.'{}'""".format(q)
    template = """{facts}\n""" + template
    prompt_template = PromptTemplate(input_variables=["facts"], template=template)
    answer_chain = LLMChain(llm=llm, prompt=prompt_template)
    total_prompt = total_prompt + template

    overall_chain = SimpleSequentialChain(chains=[assumptions_chain, fact_checker_chain, answer_chain], verbose=True)
    answer = overall_chain.run(q)

    #Logging prompt data
    if table!=None:
        table.add_data(llm.model_id, q, answer, total_prompt)

    return answer


In [25]:
from IPython.display import display, Markdown

print(f'The statement is: {q}')
display(Markdown(generate_and_print(llm, q)))

The statement is: The first woman to receive a Ph.D. in computer science was Dr. Barbara Liskov, who earned her degree from Stanford University in 1968.


> Entering new SimpleSequentialChain chain...
* The first woman to receive a Ph.D. in computer science was American.
* The first woman to receive a Ph.D. in computer science earned her degree in the 1960s.
* The first woman to receive a Ph.D. in computer science studied at Stanford University.
The first assertion is true. In 1964, American Adele Goldberg became the first woman to receive a Ph.D. in computer science. 

 The second assertion is false. The first woman to receive a Ph.D. in computer science earned her degree in the 1960s, but she did not study at Stanford University. 

 The third assertion is true. In 1961, American Barbara Liskov studied at Stanford University and became the first woman to receive a Ph.D. in computer science.
 So the answer is False

> Finished chain.


 So the answer is False

## Using wandb to capture prompts and responses

In [26]:
import wandb
WANDB_NOTEBOOK_NAME='fact_checker'
wandb.login(key=os.environ['WANDB_API_KEY'])
#wandb.login(key=WANDB_API_KEY)
#wandb.login(anonymous="allow")



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marcelcastrobr. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/marcasbr/.netrc


True

In [27]:
run = wandb.init(
    project="fakenewsevaluation", 
    job_type="generation")

# Define W&B Table to store generations
columns = ["model", "question", "answer", "prompt"]
table = wandb.Table(columns=columns)

In [28]:
from IPython.display import display, Markdown
print(q)
display(Markdown(generate_and_print(llm, q, table)))

wandb.log({"simple_generations": table})
run.finish()

The first woman to receive a Ph.D. in computer science was Dr. Barbara Liskov, who earned her degree from Stanford University in 1968.


> Entering new SimpleSequentialChain chain...
* The first woman to receive a Ph.D. in computer science was American.
* The first woman to receive a Ph.D. in computer science earned her degree in the 1960s.
* The first woman to receive a Ph.D. in computer science studied at Stanford University.
The first assertion is true. In 1964, American Adele Goldberg became the first woman to receive a Ph.D. in computer science. 

 The second assertion is false. The first woman to receive a Ph.D. in computer science earned her degree in the 1960s, but she did not study at Stanford University. 

 The third assertion is true. In 1961, American Barbara Liskov studied at Stanford University and became the first woman to receive a Ph.D. in computer science.
 So the answer is False

> Finished chain.


 So the answer is False

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
